<a href="https://colab.research.google.com/github/yasyamauchi/education/blob/main/decision_tree_BME2024_report3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2024年人工知能第3回レポート課題用  
# 国勢調査からの収入予測  
  
[Google Machine Learning: 上級コース: デシジョンフォレスト](https://developers.google.com/machine-learning/decision-forests/practice?hl=ja)より  
[YDF: getting started](https://ydf.readthedocs.io/en/latest/tutorial/getting_started/)より


YDF（Yggdrasil）を使用する．

In [ ]:
!pip install ydf -U

In [ ]:
import ydf
import numpy as np
import pandas as pd

## データの読み込み  
  
  この例ではYDFのサンプルデータセット(個人の収入と属性)を使用する．  
### データの中身  
成人(合計22792人)の収入 (**income**)が，50,000ドルより大きい(>50K)か，50,000ドル以下か(<=50K)かを，他のデータから明らかにする．  
  
* 年齢 (age)
* 就業形態 (workclass)
* final weight *意味不明 (fnlwgt)
* 学歴 (education)
* 教育年数 (education_num)
* 配偶者の有無 (marital_status)
* 職業 (occupation)  
* 夫/妻/独身等 (relationship)
* 人種 (race)
* 性 (sex)
* キャピタルゲイン[株式譲渡益等] (capital_gain)
* キャピタルロス[株式譲渡損失等] (capital_loss)
* 週当たり労働時間 (hours_per_week)
* 出生国 (native_country)
* **収入 (income)**  
  
最初の10人のデータを表示する．  
  
**この中の7番(age=32,workclass=Private...)を，のちに使用する**  

In [ ]:
path = "https://raw.githubusercontent.com/google/yggdrasil-decision-forests/main/yggdrasil_decision_forests/test_data/dataset/adult_train.csv"
dataset = pd.read_csv(path)
label = "income"

# Display the first 10 examples.
dataset.head(10)

## 訓練データと検証データに分ける  
  
訓練データと検証データに分ける．ランダムに分けるので実行するたびに結果は異なる．

**【問1】** 検証データは全体のどのくらいが適切か？  
割合を答えよ．たとえば6割の場合は0.6と答えること．

**重要！問1で答えた結果を下記のコードに記入して分割を実行する．具体的には2行目の最後の数字(0.6)を書き替える．**

In [ ]:
np.random.seed(1)
is_test = np.random.rand(len(dataset)) < 0.6 ## この0.6というのを書き替える

train_dataset = dataset[~is_test]
test_dataset = dataset[is_test]

print("Training examples: ", len(train_dataset))

print("Testing examples: ", len(test_dataset))

## モデルの学習  
  
デフォルト値で行う．学習はすぐ終わる．

In [ ]:
model = ydf.CartLearner(label=label).train(train_dataset)

## モデルの評価
  
  まず決定木を表示する．  
  Yesが上でNoが下．赤が<=50,000で青が>50,000．
   
   「ペンギン」の例では分岐条件は数字だけであったが，今回のデータでは数値以外の分岐もある．たとえば最初の分岐が**relationship(夫/妻/その他)**である(と思われる)が，その上にマウスカーソルを当てると**relationship in [Husband, Wife]**という吹き出しが出る．この場合，relationshipというデータがHusbandまたはWifeの場合はYes(上に行く)，その他の場合はNo(下に行く)という意味である．

In [ ]:
model.plot_tree()

精度を計算する．

**【問2】** 訓練データによる精度と，検証データによる精度を答えよ．

In [ ]:
train_evaluation = model.evaluate(train_dataset)
print("train accuracy:", train_evaluation.accuracy)

test_evaluation = model.evaluate(test_dataset)
print("test accuracy:", test_evaluation.accuracy)

このままでは複雑なので，木の深さを最大4とする

In [ ]:
model = ydf.CartLearner(label=label, max_depth=4).train(train_dataset)
print(model.evaluate(test_dataset).accuracy)

**【問3】** この決定木を提出する．Snipping Toolsなどで画面キャプチャするとよい．  
**【問4】** この決定木を用いて，7番の人の収入を推定する．  
*解答例：relationshipがWifeで，capital gainが5000以上で，学歴がMastersなので，収入は<=50,000と判断した*

In [ ]:
model.plot_tree()